In [13]:
import torchvision
import torch, math

#### Change the relevant elements in the model

In [14]:
model = torchvision.models.detection.fcos_resnet50_fpn(
    weights=torchvision.models.detection.FCOS_ResNet50_FPN_Weights.DEFAULT,
    weights_backbone=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)

In [15]:
model.backbone.out_channels


256

In [16]:
num_anchors = model.head.classification_head.num_anchors
num_anchors

1

In [17]:
num_class = 2
updated_cls_logits = torch.nn.Conv2d(model.backbone.out_channels, num_anchors*num_class, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(updated_cls_logits.weight, std=0.01)  # as per pytorch code
torch.nn.init.constant_(updated_cls_logits.bias, -math.log((1 - 0.01) / 0.01)) 
model.head.classification_head.cls_logits = updated_cls_logits
model.head.classification_head.num_classes = num_class


In [18]:
model.head.classification_head.cls_logits

Conv2d(256, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [19]:
model.transform = torchvision.models.detection.transform.GeneralizedRCNNTransform(min_size=512, max_size=512, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

### On how to change the number of classes in the model

[https://discuss.pytorch.org/t/object-detection-fine-tuning-model-initialisation-error/159940/4](Link)


```python
from torchvision.models.detection import fcos_resnet50_fpn, FCOS_ResNet50_FPN_Weights
import math
weights = FCOS_ResNet50_FPN_Weights.DEFAULT
model = fcos_resnet50_fpn(weights=weights)  # load an object detection model pre-trained on COCO
num_anchors = model.head.classification_head.num_anchors
model.head.classification_head.num_classes = num_class
out_channels = model.head.classification_head.conv[9].out_channels
cls_logits = torch.nn.Conv2d(out_channels, num_anchors * num_class, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))
```

In [20]:
model.eval()

FCOS(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, eps=1e

In [21]:
type(model).__name__

'FCOS'

### Load dataset into the model

In [22]:
from dataset import SyntheticImage
from pathlib import Path

In [23]:
import numpy as np

In [24]:
umr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Sythetic_Output\UoMR')
ukr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Sythetic_Output\UKR')
dir_list = [umr_dir, ukr_dir]
synthetic_image = SyntheticImage(dir_list)

In [25]:
target, image = synthetic_image[0]

In [26]:
images = [image]

In [27]:
image.shape

torch.Size([3, 512, 512])

In [28]:
outputs = model([image])

Model in evaluation mode, the output is a list of dictionaries, one for each input image. The dictionary contains the following fields:
`'boxes', 'scores','labels'`

In [29]:
outputs

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'scores': tensor([], grad_fn=<IndexBackward0>),
  'labels': tensor([], dtype=torch.int64)}]

In [30]:
detection_threshold = 0.3
STENOSIS_CLASSES = {1: 'mild', 2: 'severe'}
pred_scores = outputs[0]['scores'].detach().cpu().numpy()

# Get all the predicted bounding boxes.
pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()
# Get boxes above the threshold score.
boxes = pred_bboxes[pred_scores >= detection_threshold].astype(np.int32)
labels = outputs[0]['labels'][pred_scores >= detection_threshold]
# Get all the predicited class names.
pred_classes = [STENOSIS_CLASSES[i] for i in labels.cpu().numpy()]

Model in training mode, the output is a dict containing multiple loss with the fowllowing keys:
`'classification', 'bbox_regression', 'bbox_ctrness'`


In [36]:
import collections

In [38]:
import torch.utils.data as module_dataloader
import torch 

In [63]:
def cutomized_collate(batch):
    elem = batch[0]
    
    elem_type = type(elem)
    print(elem_type)
    if isinstance(elem, torch.Tensor):
        out = None
        if torch.utils.data.get_worker_info() is not None:
            # If we're in a background process, concatenate directly into a
            # shared memory tensor to avoid an extra copy
            numel = sum([x.numel() for x in batch])
            storage = elem.storage()._new_shared(numel)
            out = elem.new(storage)
        return torch.stack(batch, 0, out=out)

    elif isinstance(elem, float):
        return torch.tensor(batch, dtype=torch.float64)
    elif isinstance(elem, int):
        return torch.tensor(batch)
    elif isinstance(elem, collections.abc.Mapping):
        print('hello')
        return list(batch)
    elif isinstance(elem, tuple):  # namedtuple
        print(batch)
        return elem_type(*(cutomized_collate(samples) for samples in zip(*batch)))
    elif isinstance(elem, collections.abc.Sequence):
        # check to make sure that the elements in batch have consistent size
        it = iter(batch)
        elem_size = len(next(it))
        if not all(len(elem) == elem_size for elem in it):
            raise RuntimeError('each element in list of batch should be of equal size')
        transposed = zip(*batch)
        return [module_dataloader.dataloader.default_collate(samples) for samples in transposed]

In [64]:
variables = {'batch_size': 4,
    'collate_fn':cutomized_collate,
    'sampler':module_dataloader.sampler.RandomSampler(range(len(synthetic_image))),
    'drop_last':True,
    'shuffle':False}

In [65]:
real_image_dataloader = torch.utils.data.DataLoader(synthetic_image, **variables)

In [66]:
for targets, images in real_image_dataloader:
    break

<class 'tuple'>
[({'boxes': tensor([[264.5000, 416.5000, 282.5000, 434.5000]]), 'labels': tensor([0])}, tensor([[[0.4627, 0.4706, 0.4902,  ..., 0.4157, 0.3804, 0.3608],
         [0.4667, 0.4745, 0.4824,  ..., 0.4157, 0.3961, 0.3765],
         [0.4745, 0.4745, 0.4863,  ..., 0.4314, 0.4078, 0.3922],
         ...,
         [0.2000, 0.2039, 0.1961,  ..., 0.1765, 0.1804, 0.1843],
         [0.2039, 0.2039, 0.1961,  ..., 0.1765, 0.1765, 0.1804],
         [0.1922, 0.1961, 0.1961,  ..., 0.1725, 0.1725, 0.1725]],

        [[0.4627, 0.4706, 0.4902,  ..., 0.4157, 0.3804, 0.3608],
         [0.4667, 0.4745, 0.4824,  ..., 0.4157, 0.3961, 0.3765],
         [0.4745, 0.4745, 0.4863,  ..., 0.4314, 0.4078, 0.3922],
         ...,
         [0.2000, 0.2039, 0.1961,  ..., 0.1765, 0.1804, 0.1843],
         [0.2039, 0.2039, 0.1961,  ..., 0.1765, 0.1765, 0.1804],
         [0.1922, 0.1961, 0.1961,  ..., 0.1725, 0.1725, 0.1725]],

        [[0.4627, 0.4706, 0.4902,  ..., 0.4157, 0.3804, 0.3608],
         [0.4667, 0

TypeError: tuple expected at most 1 arguments, got 2

In [67]:
targets

{'boxes': tensor([[[187.5000, 363.0000, 205.5000, 381.0000]],
 
         [[261.0000, 222.0000, 281.0000, 242.0000]],
 
         [[332.5000, 313.5000, 350.5000, 331.5000]],
 
         [[156.0000, 329.0000, 174.0000, 347.0000]]]),
 'labels': tensor([[1],
         [1],
         [1],
         [1]])}

In [69]:
eg_list = list(list(targets[key]) for key in targets)

In [77]:
len(targets)

2

In [72]:
target_dict_list = [dict(zip(targets.keys(), [eg_list[i][j] for i in range(len(targets))])) for j in range(len(eg_list[0]))]

In [73]:
target_dict_list

[{'boxes': tensor([[187.5000, 363.0000, 205.5000, 381.0000]]),
  'labels': tensor([1])},
 {'boxes': tensor([[261., 222., 281., 242.]]), 'labels': tensor([1])},
 {'boxes': tensor([[332.5000, 313.5000, 350.5000, 331.5000]]),
  'labels': tensor([1])},
 {'boxes': tensor([[156., 329., 174., 347.]]), 'labels': tensor([1])}]

In [78]:
targets = [{k: v.to('cpu') for k, v in t.items()} for t in target_dict_list]

In [79]:
targets

[{'boxes': tensor([[187.5000, 363.0000, 205.5000, 381.0000]]),
  'labels': tensor([1])},
 {'boxes': tensor([[261., 222., 281., 242.]]), 'labels': tensor([1])},
 {'boxes': tensor([[332.5000, 313.5000, 350.5000, 331.5000]]),
  'labels': tensor([1])},
 {'boxes': tensor([[156., 329., 174., 347.]]), 'labels': tensor([1])}]

In [74]:
model.train()
with torch.no_grad():
    outputs = model([image]*4, target_dict_list)

In [75]:
target

{'boxes': tensor([[249.5000, 412.5000, 269.5000, 432.5000]]),
 'labels': tensor([1])}

In [76]:
outputs

{'classification': tensor(1.1089),
 'bbox_regression': tensor(0.9021),
 'bbox_ctrness': tensor(0.8343)}

In [33]:
targets = {'boxes': torch.tensor([[[264.5000, 430.0000, 284.5000, 450.0000]],
        [[117.0000, 245.0000, 135.0000, 263.0000]]]), 'labels': torch.tensor([[0],
        [0]])}

In [35]:
list(zip(*targets))

[('b', 'l'), ('o', 'a'), ('x', 'b'), ('e', 'e'), ('s', 'l')]

In [32]:
outputs

{'classification': tensor(1.0974),
 'bbox_regression': tensor(0.8437),
 'bbox_ctrness': tensor(0.7195)}

In [27]:
[target]

[{'boxes': tensor([[249.5000, 412.5000, 269.5000, 432.5000]]),
  'labels': tensor([1])}]

In [26]:
outputs

{'classification': tensor(1.1333, grad_fn=<DivBackward0>),
 'bbox_regression': tensor(0.8437, grad_fn=<DivBackward0>),
 'bbox_ctrness': tensor(0.7195, grad_fn=<DivBackward0>)}

In [52]:
losses = sum(loss for loss in outputs.values())

In [53]:
losses

tensor(2.6914, grad_fn=<AddBackward0>)

#### Check the number of examples in the dataset

In [80]:
from dataset import SyntheticImage
from pathlib import Path

In [81]:
umr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Sythetic_Output\UoMR')
ukr_dir=Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Sythetic_Output\UKR')
synthetic_image_umr = SyntheticImage([umr_dir])
synthetic_image_ukr = SyntheticImage([ukr_dir])

In [82]:
len(synthetic_image_umr)

1357

In [83]:
len(synthetic_image_ukr)

281

### Check the Real Image Dataset

In [4]:
from dataset import RealImage
from pathlib import Path

In [5]:
umr_dir = Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Real_Image\UMR\Full')
ukr_dir = Path(r'Z:\Projects\Angiogram\Data\Processed\Zijun\Synthetic\Real_Image\UKR\Full')
dir_list = [umr_dir, ukr_dir]

In [6]:
real_image_data = RealImage(dir_list)

In [8]:
real_image_data[0].shape

torch.Size([3, 512, 512])

In [10]:
import torchvision
print(torchvision.__version__)

0.14.1+cpu


In [11]:
import torch
print(torch.__version__)

1.13.1+cpu
